In [13]:
from numpy import mean
import pandas as pd
import boto3
import os
import time
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from util.preprocess import *
import trp
import pickle
# load saved model
with open('model_pkl' , 'rb') as f:
    model = pickle.load(f)

[07:37:36] WARNING: ../src/learner.cc:1040: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[07:37:36] WARNING: ../src/learner.cc:749: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.
[07:37:36] WARNING: ../src/learner.cc:438: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details a

In [19]:
# Retrieve the list of existing buckets
s3 = boto3.client('s3')
response = s3.list_buckets()

# Output the bucket names
for bucket in response['Buckets']:
    print(bucket);


{'Name': 'amplify-mediscan-dev-203055-deployment', 'CreationDate': datetime.datetime(2022, 10, 13, 15, 1, 3, tzinfo=tzlocal())}
{'Name': 'mediscan-aws-hackathon', 'CreationDate': datetime.datetime(2022, 10, 13, 12, 42, 15, tzinfo=tzlocal())}
{'Name': 'mediscand65087d67ba848f6936468215a95f8d2203055-dev', 'CreationDate': datetime.datetime(2022, 10, 14, 11, 53, 36, tzinfo=tzlocal())}
{'Name': 'sagemaker-ap-south-1-134811648823', 'CreationDate': datetime.datetime(2022, 10, 13, 18, 0, 15, tzinfo=tzlocal())}


In [15]:
def extract_text(textractJobId,response):
    pages = []

    time.sleep(5)
    textract = boto3.client('textract')
    response = textract.get_document_analysis(JobId=textractJobId)

    pages.append(response)

    nextToken = None
    if('NextToken' in response):
        nextToken = response['NextToken']

    while(nextToken):
        time.sleep(5)
        textract = boto3.client('textract')
        response = textract.get_document_analysis(JobId=textractJobId, NextToken=nextToken)

        pages.append(response)
        print("Resultset page recieved: {}".format(len(pages)))
        nextToken = None
        if('NextToken' in response):
            nextToken = response['NextToken']
    
    return pages 

In [20]:
def extractTextract(bucket,textractObjectName):
    textract = boto3.client('textract')
    response = textract.start_document_analysis(
        DocumentLocation={
            'S3Object': {
            'Bucket': 'mediscan-aws-hackathon',
            'Name': 'sample_report_1.pdf',

        }
            },
        FeatureTypes=[
            'TABLES',
        ]
        )

    textractJobId = response["JobId"]
    print('job id is: ',textractJobId)
    time.sleep(15)
    response = textract.get_document_analysis(JobId=textractJobId)
    status = response["JobStatus"]

    while(status == "IN_PROGRESS"):
        time.sleep(5)
        response = textract.get_document_analysis(JobId=textractJobId)
        status = response["JobStatus"]
        print("Textract Job status: {}".format(status))
    
    pages=extract_text(textractJobId,response)
    doc = trp.Document(pages)
    return doc

In [21]:
def extractMedical(doc):
    maxLength = 10000
    comprehendResponse = []
    comprehend_medical_client = boto3.client(service_name='comprehendmedical', region_name='ap-southeast-2')
    for page in doc.pages:
        pageText = page.text
    
        for i in range(0, len(pageText), maxLength):
            response = comprehend_medical_client.detect_entities_v2(Text=pageText[0+i:maxLength+i])
            comprehendResponse.append(response)
        patient_string = ""
        
    #df_cm=extractMC_v2(comprehendResponse[0])
    return comprehendResponse

#############################################################################################
############# functions to convert all medical conditions to 1 record ########################

In [23]:
# pipeline for new pdf
#extract data from textract
fileName =  'sample_report_1.pdf'
textractObjectName = os.path.join('public', fileName)
print("EHR file to be processed is at ", textractObjectName)
textract = boto3.client('textract')
doc=extractTextract(bucket,textractObjectName)
#extract data from comprehend medical
comprehendResponse=extractMedical(doc)
df_cm=extractMC_v2(comprehendResponse[0])
#Organize the extracted json file into dataframe
mclist, df_cm2=retrieve_mcList(df_cm, nFeature=20,threshold=0.9)
df_cm2=df_mc_generator_slim(df_cm2)
df_cm2
#Prediction with the endpoint
sample_pred = model.predict()
print(sample_pred)

EHR file to be processed is at  public/sample_report_1.pdf
job id is:  0d92f30b914828f218928a7763f2d91ea85d477bddc31b095d8b0bd4216dcc5d


UnboundLocalError: local variable 'textract' referenced before assignment